## Imports

In [3]:
import pandas as pd

import surprise
from surprise.prediction_algorithms import *
import pandas as pd
import numpy as np
import datetime as dt

from surprise import Dataset
from surprise import Reader

from surprise.prediction_algorithms import knns
from surprise.similarities import cosine, msd, pearson
from surprise import accuracy
from surprise.model_selection import GridSearchCV
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate

from sklearn import model_selection as smodel

from nltk.tokenize import RegexpTokenizer

import pickle 

## EDA

In [4]:
movies = pd.read_csv('../../ml-latest-small/movies.csv')

In [5]:
link = pd.read_csv('../../ml-latest-small/links.csv')

In [6]:
rating = pd.read_csv('../../ml-latest-small/ratings.csv')

In [7]:
tags = pd.read_csv('../../ml-latest-small/tags.csv')

In [8]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [9]:
movies.isna().sum()

movieId    0
title      0
genres     0
dtype: int64

In [10]:
movies.title.value_counts()

Eros (2004)                                              2
Emma (1996)                                              2
Saturn 3 (1980)                                          2
War of the Worlds (2005)                                 2
Confessions of a Dangerous Mind (2002)                   2
                                                        ..
Anacleto: Agente secreto (2015)                          1
Evil Dead II (Dead by Dawn) (1987)                       1
The Walk (2015)                                          1
Three Musketeers, The (1993)                             1
Alias Betty (Betty Fisher et autres histoires) (2001)    1
Name: title, Length: 9737, dtype: int64

In [11]:
#Check to see if there are any duplicate titles
movies.title.duplicated().sum()

5

In [12]:
#Drop the 5 duplicated movie titles
movies.drop_duplicates(subset='title', inplace=True)

In [13]:
#Sanity check to ensure all duplicates were dropped from title column

movies.title.duplicated().sum()

0

In [14]:
movies[movies.title.duplicated() == True]

,movieId,title,genres


In [15]:
#Check length of DataFrame

len(movies)

9737

In [16]:
# Split title column into two new columns: Title and year 

movies['Title'] = movies['title'].str.split('(', n=1, expand=True)[0]

movies['year_released'] = movies['title'].str.split('(', n=1, expand=True)[1]

In [17]:
# Drop original column title

movies.drop(columns='title', inplace=True)

In [18]:
# Sanity Check 
movies

,movieId,genres,Title,year_released
0,1,Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995)
1,2,Adventure|Children|Fantasy,Jumanji,1995)
2,3,Comedy|Romance,Grumpier Old Men,1995)
3,4,Comedy|Drama|Romance,Waiting to Exhale,1995)
4,5,Comedy,Father of the Bride Part II,1995)
...,...,...,...,...
9737,193581,Action|Animation|Comedy|Fantasy,Black Butler: Book of the Atlantic,2017)
9738,193583,Animation|Comedy|Fantasy,No Game No Life: Zero,2017)
9739,193585,Drama,Flint,2017)
9740,193587,Action|Animation,Bungo Stray Dogs: Dead Apple,2018)


In [19]:
# Format year_released column

movies['year_released'] = movies.year_released.str.replace(')', '')

In [20]:
# Sanity Check to ensure formatting was completed
movies

,movieId,genres,Title,year_released
0,1,Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995
1,2,Adventure|Children|Fantasy,Jumanji,1995
2,3,Comedy|Romance,Grumpier Old Men,1995
3,4,Comedy|Drama|Romance,Waiting to Exhale,1995
4,5,Comedy,Father of the Bride Part II,1995
...,...,...,...,...
9737,193581,Action|Animation|Comedy|Fantasy,Black Butler: Book of the Atlantic,2017
9738,193583,Animation|Comedy|Fantasy,No Game No Life: Zero,2017
9739,193585,Drama,Flint,2017
9740,193587,Action|Animation,Bungo Stray Dogs: Dead Apple,2018


In [21]:
link

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9737,193581,5476944,432131.0
9738,193583,5914996,445030.0
9739,193585,6397426,479308.0
9740,193587,8391976,483455.0


In [22]:
rating

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [23]:
#converted timestamp to datetime
rating['timestamp'] = pd.to_datetime(rating['timestamp'], unit='s')

In [24]:
rating.rating.value_counts(normalize=True)

4.0    0.265957
3.0    0.198808
5.0    0.131015
3.5    0.130271
4.5    0.084801
2.0    0.074884
2.5    0.055040
1.0    0.027877
1.5    0.017762
0.5    0.013586
Name: rating, dtype: float64

In [25]:
rating.duplicated().sum()

0

In [26]:
rating.isna().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [27]:
tags

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
...,...,...,...,...
3678,606,7382,for katie,1171234019
3679,606,7936,austere,1173392334
3680,610,3265,gun fu,1493843984
3681,610,3265,heroic bloodshed,1493843978


In [28]:
movie_rating = movies.merge(rating, on='movieId', how='outer')

In [29]:
movie_rating

,movieId,genres,Title,year_released,userId,rating,timestamp
0,1,Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995,1.0,4.0,2000-07-30 18:45:03
1,1,Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995,5.0,4.0,1996-11-08 06:36:02
2,1,Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995,7.0,4.5,2005-01-25 06:52:26
3,1,Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995,15.0,2.5,2017-11-13 12:59:30
4,1,Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995,17.0,4.5,2011-05-18 05:28:03
...,...,...,...,...,...,...,...
100849,64997,NaN,NaN,NaN,68.0,2.5,2008-12-28 20:55:15
100850,144606,NaN,NaN,NaN,111.0,4.0,2018-01-31 23:27:37
100851,147002,NaN,NaN,NaN,318.0,4.0,2017-08-08 15:45:52
100852,26958,NaN,NaN,NaN,509.0,3.5,2015-07-04 17:42:33


In [30]:
movie_rating.dropna(inplace=True)

In [31]:
movie_rating['genres']=[row.strip().lower().replace('|',',') for row in movie_rating['genres']]

In [32]:
movie_rating['userId'].nunique()

610

In [33]:
stats = movie_rating.filter(['rating', 'timestamp']).describe()
stats

,rating
count,100813.000000
mean,3.501557
std,1.042494
min,0.500000
25%,3.000000
50%,3.500000
75%,4.000000
max,5.000000


In [34]:
movie_rating

,movieId,genres,Title,year_released,userId,rating,timestamp
0,1,"adventure,animation,children,comedy,fantasy",Toy Story,1995,1.0,4.0,2000-07-30 18:45:03
1,1,"adventure,animation,children,comedy,fantasy",Toy Story,1995,5.0,4.0,1996-11-08 06:36:02
2,1,"adventure,animation,children,comedy,fantasy",Toy Story,1995,7.0,4.5,2005-01-25 06:52:26
3,1,"adventure,animation,children,comedy,fantasy",Toy Story,1995,15.0,2.5,2017-11-13 12:59:30
4,1,"adventure,animation,children,comedy,fantasy",Toy Story,1995,17.0,4.5,2011-05-18 05:28:03
...,...,...,...,...,...,...,...
100843,193581,"action,animation,comedy,fantasy",Black Butler: Book of the Atlantic,2017,184.0,4.0,2018-09-16 14:44:42
100844,193583,"animation,comedy,fantasy",No Game No Life: Zero,2017,184.0,3.5,2018-09-16 14:52:25
100845,193585,drama,Flint,2017,184.0,3.5,2018-09-16 14:56:45
100846,193587,"action,animation",Bungo Stray Dogs: Dead Apple,2018,184.0,3.5,2018-09-16 15:00:21


In [35]:
# movie_rating['genres'] = movie_rating['genres'].to_list()

In [36]:
# pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
# tokenizer = RegexpTokenizer(pattern)
# movie_rating['genres'] = [tokenizer.tokenize(token) for token in movie_rating['genres']]

## Split

In [37]:
train, test = smodel.train_test_split(movie_rating, random_state=42)

In [38]:
reader = Reader(rating_scale=(1, 5))
train_data = Dataset.load_from_df(train[['userId', 'movieId', 'rating']], reader)

In [49]:
type(train_data)

surprise.dataset.DatasetAutoFolds

In [39]:
test_data = Dataset.load_from_df(test[['userId', 'movieId', 'rating']], reader)

In [40]:
type (test_data)

surprise.dataset.DatasetAutoFolds

In [41]:
movie_rating_df = movie_rating[['userId', 'movieId', 'rating']]
# movie_rating_df.columns = ['userId', 'movieId', 'raw_ratings']

In [42]:
reader = Reader(rating_scale=(1, 5))
movie_rating_ds = Dataset.load_from_df(movie_rating_df, reader)

In [43]:
trainset = movie_rating_ds.build_full_trainset()

In [44]:
type(movie_rating_ds)

surprise.dataset.DatasetAutoFolds

In [45]:
# train_data, test_data = smodel.train_test_split(movie_rating_df, random_state=42)
trainset, testset = train_test_split(movie_rating_ds, random_state=42)

In [46]:
type(trainset)

surprise.trainset.Trainset

## Tuning

### KNNBasics

In [55]:
param_grid = {'k':[10, 50, 100],'min_k': [1, 5, 10]}
base_model = GridSearchCV(KNNBasic,param_grid=param_grid,joblib_verbose=5)
base_model.fit(train_data)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Computing the msd similarity matrix...
Done computing similarity matrix.


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s


Computing the msd similarity matrix...
Done computing similarity matrix.


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.0s remaining:    0.0s


...

[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    1.6s remaining:    0.0s


Computing the msd similarity matrix...
Done computing similarity matrix.


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    2.1s remaining:    0.0s


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:   28.6s finished


In [ ]:
base_model.best_params

In [ ]:
param_grid = {'k':[5, 10, 15],'min_k': [1, 5, 10]}
base_model = GridSearchCV(KNNBasic,param_grid=param_grid,joblib_verbose=5)
base_model.fit(train_data)

In [ ]:
base_model.best_params

In [ ]:
param_grid = {'k':[10, 15, 20],'min_k': [1, 5, 10]}
base_model = GridSearchCV(KNNBasic,param_grid=param_grid,joblib_verbose=5)
base_model.fit(train_data)

In [ ]:
base_model.best_params

Best Params for KNNBasics is k=15, min_k = 5.

## Cross Validation

In [ ]:
#instantiate KNN model 
model1=KNNBasic(k=15, min_k=5)


In [ ]:
#instaniatite and perform cross val with training data
cross_validate(model1, train_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

In [ ]:
#use model to predict
model1.predict(uid=10, iid = 20)
#this model just predicted what user 10 would predict movie 20

looking at the output above we can determine that our model predicted that use 10 would rate movie 20 a 2.5 give or take our RSME which is .95.

## Models

### SVD

In [51]:
model2 = SVD()

In [54]:
model2.fit(trainset)

In [53]:
cross_validate(model2, train_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8770  0.8741  0.8801  0.8782  0.8940  0.8807  0.0069  
MAE (testset)     0.6753  0.6766  0.6805  0.6748  0.6859  0.6786  0.0041  
Fit time          3.13    3.09    3.10    3.13    3.12    3.11    0.02    
Test time         0.11    0.07    0.11    0.07    0.07    0.09    0.02    


{'test_rmse': array([0.87703689, 0.87414758, 0.88010125, 0.87816621, 0.89404373]),
 'test_mae': array([0.67527705, 0.67659572, 0.68049581, 0.67478795, 0.68587177]),
 'fit_time': (3.1337499618530273,
  3.0947208404541016,
  3.096944808959961,
  3.1263210773468018,
  3.122912883758545),
 'test_time': (0.11261200904846191,
  0.07059788703918457,
  0.11052203178405762,
  0.0700528621673584,
  0.07198596000671387)}

In [ ]:
param_grid = {'n_factors':[10,20,50],'n_epochs': [5, 10, 15], 'lr_all': [0.002,0.005,0.01],
             'reg_all': [0.2,0.4,0.6]}
gs_model2 = GridSearchCV(SVD,param_grid=param_grid,joblib_verbose=5)
gs_model2.fit(train_data)

In [ ]:
gs_model2.best_params

In [ ]:
param_grid = {'n_factors':[25, 50, 75],'n_epochs': [15, 30, 45], 'lr_all': [0.01,0.05,0.1],
             'reg_all': [0.1,0.2,0.3]}
gs_model2 = GridSearchCV(SVD,param_grid=param_grid,joblib_verbose=5)
gs_model2.fit(train_data)

In [ ]:
gs_model2.best_params

In [ ]:
cross_validate(SVD(n_factors=75,n_epochs=45,lr_all=0.01,reg_all=0.1), train_data, measures=['RMSE', 'MAE'], 
               cv=5, verbose=True)

In [ ]:
param_grid = {'n_factors':[75, 100, 125],'n_epochs': [45, 60, 75], 'lr_all': [.005, 0.01, .015],
             'reg_all': [.05, 0.1, 0.15]}
gs_model2 = GridSearchCV(SVD,param_grid=param_grid,joblib_verbose=5)
gs_model2.fit(train_data)

In [ ]:
gs_model2.best_params

At some point we have to stop. If we have more time we will run another grid search for the model above. 
Best Params for SVD are n_factors=125, n_epochs= 45, lr_all= .015, reg_all= .1.

In [120]:
model = SVD(n_factors=125,n_epochs=45,lr_all=0.015,reg_all=0.1)

cross_validate(model, train_data, measures=['RMSE', 'MAE'], 
               cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8574  0.8568  0.8605  0.8679  0.8613  0.8608  0.0040  
MAE (testset)     0.6589  0.6610  0.6595  0.6666  0.6599  0.6612  0.0028  
Fit time          8.38    8.37    8.39    8.39    8.33    8.37    0.02    
Test time         0.07    0.07    0.07    0.07    0.07    0.07    0.00    


{'test_rmse': array([0.85737477, 0.85677494, 0.86050478, 0.86794986, 0.861253  ]),
 'test_mae': array([0.65891036, 0.66097398, 0.65953158, 0.66658012, 0.65986547]),
 'fit_time': (8.3762788772583,
  8.371620893478394,
  8.385881900787354,
  8.38858699798584,
  8.330207109451294),
 'test_time': (0.06965517997741699,
  0.07026791572570801,
  0.07081794738769531,
  0.0722508430480957,
  0.07192397117614746)}

Best model so far with an RMSE of .86.

### NMF

In [ ]:
model3 = NMF()

In [ ]:
cross_validate(model3, train_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

In [ ]:
param_grid = {'n_factors':[10,15,20],'n_epochs': [25, 50, 75]}
gs_model3 = GridSearchCV(NMF,param_grid=param_grid,joblib_verbose=5)
gs_model3.fit(train_data)

In [ ]:
gs_model3.best_params

In [ ]:
cross_validate(NMF(n_factors=10,n_epochs=25), train_data, measures=['RMSE', 'MAE'], 
               cv=5, verbose=True,  n_jobs= -2)

In [ ]:
param_grid = {'n_factors':[3,5,10],'n_epochs': [15, 20, 25]}
gs_model3 = GridSearchCV(NMF,param_grid=param_grid,joblib_verbose=5)
gs_model3.fit(train_data)

In [ ]:
gs_model3.best_params

In [ ]:
# Instantiate reader the same convention as a sklearn class
reader = Reader(rating_scale=(1, 5))
# Load user_data from imported dataframe specifiyng which columns to use
user_data = Dataset.load_from_df(df_rev5[['reviewerID', 'asin', 'overall']], reader)

# Perform an appropriate split for the recommendation models
trainset, testset = train_test_split(user_data, test_size=0.2, random_state=42)

## Final Model Evaluation 

In [121]:
model.test(testset)

[Prediction(uid=263.0, iid=5617, r_ui=4.5, est=4.0701659116538655, details={'was_impossible': False}),
 Prediction(uid=414.0, iid=5425, r_ui=4.0, est=3.8273069295121105, details={'was_impossible': False}),
 Prediction(uid=380.0, iid=7293, r_ui=3.0, est=3.3625452150084816, details={'was_impossible': False}),
 Prediction(uid=76.0, iid=2628, r_ui=2.0, est=2.136888052480864, details={'was_impossible': False}),
 Prediction(uid=608.0, iid=2541, r_ui=3.0, est=2.7813103727121744, details={'was_impossible': False}),
 Prediction(uid=79.0, iid=858, r_ui=5.0, est=4.767642781229847, details={'was_impossible': False}),
 Prediction(uid=290.0, iid=2391, r_ui=4.0, est=4.1953657659279315, details={'was_impossible': False}),
 Prediction(uid=599.0, iid=6217, r_ui=1.5, est=2.3791611615348884, details={'was_impossible': False}),
 Prediction(uid=322.0, iid=2302, r_ui=3.5, est=3.146757897676017, details={'was_impossible': False}),
 Prediction(uid=226.0, iid=3081, r_ui=2.5, est=3.2563972341102048, details={'wa

## Deployment 

In [ ]:
train['est_rating_user10'] = train['movieId'].apply(lambda x: model3.predict(train.userId[10], x).est)
train.sort_values(by='est_rating_user10', ascending=False, inplace=True)

In [ ]:
train

In [ ]:
user = input('userId: ')
genre = input('What genres are you interested in? ')
num_recs = input('How many recomendations would you like? ')

In [ ]:
train.info()

In [ ]:
genre

In [ ]:
genre_df = train[train['genres'].str.contains(genre)]

In [ ]:
genre_df

In [ ]:
movies['est_rating_user10'] = train['movieId'].apply(lambda x: model3.predict(uid=user, iid=x).est)
movies.sort_values(by='est_rating_user10', ascending=False, inplace=True) 

In [ ]:
movies #['Title'].head(int(num_recs))

In [ ]:
movies['est_rating_user10'].value_counts()

In [ ]:
movies['Title'].head(int(num_recs))

In [ ]:
movie_rating

## Deployment 2.0

In [ ]:
user = input('userId: ')
genre = input('What genres are you interested in? ')
num_recs = input('How many recomendations would you like? ')

In [ ]:
df_user = rating_review.reset_index('userId')
df_user.drop(columns=['Title', 'year_released', 'timestamp', 'genres', ], inplace=True)
df_user.head()

In [ ]:
reader = Reader(rating_scale=(1, 5))
full_data = Dataset.load_from_df(rating_review[['userId', 'movieId', 'rating']], reader)

In [ ]:
final_model.fit(full_data)

In [ ]:
#Function
def Stuff():
    user = input('userId: ')
    genre = input('What genres are you interested in? ')
    num_recs = input('How many recomendations would you like? ')
    sub1 = user full_data['']





